In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

In [2]:
from src.smnistp import SplitMNISTPreprocessed

benchmark = SplitMNISTPreprocessed(
    n_experiences=5,
)

100%|██████████| 235/235 [00:22<00:00, 10.51it/s]


> c:\users\sasha\repos\github\dccr\src\experiments\11_comparison\src\smnistp.py(132)SplitMNISTPreprocessed()
    131     # Use the training statistics for transforming the test dataset
--> 132     features_test = fe.process(mnist_test)
    133     mnist_test.data = features_test

tensor([[0.4887, 0.3541, 0.3955,  ..., 0.5188, 0.4986, 0.2550],
        [0.3667, 0.2150, 0.1747,  ..., 0.3803, 0.2613, 0.4864],
        [0.9975, 0.6258, 0.0326,  ..., 0.0934, 0.4712, 0.3824],
        ...,
        [0.2132, 0.1405, 0.1086,  ..., 0.9911, 0.8104, 0.2462],
        [0.1339, 0.0425, 0.0904,  ..., 0.2756, 0.4244, 0.4345],
        [0.1198, 0.0799, 0.0699,  ..., 0.0379, 0.0281, 0.0622]])
torch.Size([60000, 49])


In [3]:
# from avalanche.benchmarks import SplitMNIST
# from avalanche.benchmarks.classic import SplitCIFAR10

# Benchmark creation
# benchmark = SplitMNIST(
#     n_experiences=5,
#     # n_experiences=2,
#     # n_experiences=10,
#     # train_transform=trans,
#     # eval_transform=trans,
# )
# benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)
from src.ddvfa import DDVFAStrategy

# Create the Strategy Instance
cl_strategy = DDVFAStrategy(projectdir())

# Training Loop
print('Starting experiment...')

results = []
for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    results.append(cl_strategy.eval(benchmark.test_stream[exp_id]))
    # results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience  0
12080


  0%|          | 0/48 [00:00<?, ?it/s]


JuliaError: Exception 'MethodError: no method matching train!(::DDVFA, ::Array{Float32, 3}; y=[9, 3, 3, 3, 9, 9, 3, 3, 9, 9  …  9, 9, 9, 3, 9, 9, 3, 9, 9, 3])
Closest candidates are:
  train!(::ART, !Matched::AbstractMatrix{T} where T<:Real; y, preprocessed) at C:\Users\Sasha\.julia\packages\AdaptiveResonance\ZV3k7\src\ART\common.jl:21
  train!(::DDVFA, !Matched::AbstractVector{T} where T<:Real; y, preprocessed) at C:\Users\Sasha\.julia\packages\AdaptiveResonance\ZV3k7\src\ART\distributed\modules\DDVFA.jl:265' occurred while calling julia code:
train!(art, features, y=labels)

In [11]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import ipdb
# ipdb.set_trace()
for exp_id, experience in enumerate(benchmark.train_stream):
    train_dataset = experience.dataset
    train_data_loader = DataLoader(
        dataset=train_dataset,
        pin_memory=True,
        # batch_size=90,
        batch_size=256,
    )
    print(experience.dataset.__len__())
    for mb in tqdm(train_data_loader):
        data, labels, tasks = mb
        ipdb.set_trace()
        # jl.features = self.ext_features(data)
        # jl.features = data.numpy().transpose()
        # jl.labels = labels.numpy()
        # # ipdb.set_trace()
        # jl.eval("train!(art, features, y=labels)")

12080


  0%|          | 0/48 [00:00<?, ?it/s]

> c:\users\sasha\appdata\local\temp\ipykernel_27684\4086978925.py(14)<module>()

tensor([[[0.0013, 0.0014, 0.0012,  ..., 0.0039, 0.0028, 0.0009]],

        [[0.0007, 0.0006, 0.0003,  ..., 0.0016, 0.0024, 0.0012]],

        [[0.0015, 0.0022, 0.0027,  ..., 0.0011, 0.0007, 0.0004]],

        ...,

        [[0.0017, 0.0016, 0.0006,  ..., 0.0015, 0.0004, 0.0005]],

        [[0.0006, 0.0003, 0.0004,  ..., 0.0002, 0.0002, 0.0004]],

        [[0.0007, 0.0010, 0.0031,  ..., 0.0038, 0.0035, 0.0028]]])
torch.Size([256, 1, 49])
torch.Size([256, 1, 49])


  0%|          | 0/48 [00:17<?, ?it/s]


In [7]:
benchmark.train_stream.__getitem__(0).dataset[0][0].shape

torch.Size([1, 49])

In [5]:
%debug

> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\julia\core.py(609)check_exception()
    607         else:
    608             exception = sprint(showerror, self._as_pyobj(res))
--> 609         raise JuliaError(u'Exception \'{}\' occurred while calling julia code:\n{}'
    610                          .format(exception, src))
    611 

*** NameError: name 'jl' is not defined
> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\julia\core.py(555)_call()
    553         # logger.debug("_call(%s)", src)
    554         ans = self.api.jl_eval_string(src.encode('utf-8'))
--> 555         self.check_exception(src)
    556 
    557         return ans

> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\julia\core.py(627)eval()
    625         if src is None:
    626             return None
--> 627         ans = self._call(src)
    628         if not ans:
    629             return None

*** NameError: name 'jl' is not defined
> c:\users\sasha\repos\github\dccr\src\exp

In [6]:
benchmark.test_stream[1].classes_in_this_experience
# benchmark.train_stream[1].classes_in_this_experience

[1, 4]

In [4]:
results

[0.9266000859106529, 0.0]

In [14]:
print(jl.eval("AdaptiveResonance.get_n_weights(art)"))
print(jl.eval("size(art.F2)"))

12862
(12862,)


In [ ]:
a = [1,1,2,3]
b = [1,2,2,3]
from sklearn.metrics import accuracy_score
from statistics import mean

# mean(a)
# accuracy_score(a, b)

In [ ]:
jl.eval("a = 1234")
print(jl.a)

1234
